<a href="https://colab.research.google.com/github/agupte87/LearningPyspark/blob/master/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd ..
%mkdir spark

/


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -P /spark/ https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

--2021-07-27 15:09:50--  https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 140.211.166.134, 64.50.233.100, 64.50.236.52, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|140.211.166.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224762914 (214M) [application/x-gzip]
Saving to: ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’

spark-3.0.3-bin-had 100%[===================>] 214.35M  18.2MB/s    in 21s     

2021-07-27 15:10:11 (10.1 MB/s) - ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’ saved [224762914/224762914]



In [4]:
!tar xf /spark/spark-3.0.3-bin-hadoop3.2.tgz 
%mv spark-3.0.3-bin-hadoop3.2 /spark/

In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/spark/spark-3.0.3-bin-hadoop3.2"

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [8]:
%pwd

'/'

In [9]:
#%cd /content/
#%mkdir /chapterData 
#%cd /chapterData
#%mkdir flight-data
#%cd flight-data/


In [10]:
#%mv /chapterData/flight-data /content/

mv: cannot stat '/chapterData/flight-data': No such file or directory


In [11]:
!wget -P /content/chapterData/flight-data/ https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/csv/2015-summary.csv


--2021-07-27 15:10:24--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/csv/2015-summary.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7080 (6.9K) [text/plain]
Saving to: ‘/content/chapterData/flight-data/2015-summary.csv’

2015-summary.csv    100%[===================>]   6.91K  --.-KB/s    in 0s      

2021-07-27 15:10:25 (25.9 MB/s) - ‘/content/chapterData/flight-data/2015-summary.csv’ saved [7080/7080]



In [12]:
!wget -P /content/chapterData/flight-data/ https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/json/2015-summary.json

--2021-07-27 15:10:25--  https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/json/2015-summary.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21368 (21K) [text/plain]
Saving to: ‘/content/chapterData/flight-data/2015-summary.json’

2015-summary.json   100%[===================>]  20.87K  --.-KB/s    in 0.001s  

2021-07-27 15:10:25 (22.8 MB/s) - ‘/content/chapterData/flight-data/2015-summary.json’ saved [21368/21368]



In [13]:
from pyspark.sql.types import  StructField, StructType, StringType, LongType
myManualSchema = StructType([StructField("DEST_COUNTRY_NAME", StringType(), True),StructField("ORIGIN_COUNTRY_NAME", StringType(),True), StructField("Count", LongType(),False, metadata={"hello":"world"})])

In [14]:
df = spark.read.format("json")\
.schema(myManualSchema).\
load("/content/chapterData/flight-data/2015-summary.json")

In [15]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', Count=None)

In [16]:
from pyspark.sql import Row
myRow = Row("Aditi", "Gupte", 34)

In [17]:
myRow[2]

34

In [18]:
df = spark.read.format("json")\
.load("/content/chapterData/flight-data/2015-summary.json")

In [19]:
df.createOrReplaceTempView("dfTable")

In [20]:
from pyspark.sql.types import StructType, StructField, LongType , StringType
from pyspark.sql import Row
myManualSchema1 = StructType([StructField ("Some", StringType(), True),StructField ("Sample", StringType(), True), StructField ("Columns",LongType(), False)  ])

In [21]:
myRow = Row("Hello",None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema1)
myDf.show()

+-----+------+-------+
| Some|Sample|Columns|
+-----+------+-------+
|Hello|  null|      1|
+-----+------+-------+



In [22]:
df

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [23]:

df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(1)

+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
+-----------------+-------------------+
only showing top 1 row



In [24]:
from pyspark.sql.functions import expr, col, column
df.select(
    expr("DEST_COUNTRY_NAME"),
    col("DEST_COUNTRY_NAME"),
    column("DEST_COUNTRY_NAME")
).show(2)

+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [25]:
df.select(expr("DEST_COUNTRY_NAME as Destination_Country")\
        #  .alias("DEST_COUNTRY_NAME")\
          )\
          .show(2)

+-------------------+
|Destination_Country|
+-------------------+
|      United States|
|      United States|
+-------------------+
only showing top 2 rows



In [26]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName","DEST_COUNTRY_NAME").show(2)


+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [27]:
df.selectExpr("*", "(DEST_COUNTRY_NAME =  ORIGIN_COUNTRY_NAME ) as withinCountry").show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [28]:
df.selectExpr("count(distinct(DEST_COUNTRY_NAME))", "sum(count)").show(2)

+---------------------------------+----------+
|count(DISTINCT DEST_COUNTRY_NAME)|sum(count)|
+---------------------------------+----------+
|                              132|    453316|
+---------------------------------+----------+



In [29]:
from pyspark.sql.functions import  lit
df.select(expr("*"), lit("1").alias("One")).show(10)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
|    United States|            Ireland|  344|  1|
|            Egypt|      United States|   15|  1|
|    United States|              India|   62|  1|
|    United States|          Singapore|    1|  1|
|    United States|            Grenada|   62|  1|
|       Costa Rica|      United States|  588|  1|
|          Senegal|      United States|   40|  1|
|          Moldova|      United States|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 10 rows



In [30]:
df.withColumn("One", lit(1)).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



In [31]:
df.withColumn("withinCountry",expr("DEST_COUNTRY_NAME == ORIGIN_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [32]:
df.withColumn("Destination_country",expr("DEST_COUNTRY_NAME")).show(2)

+-----------------+-------------------+-----+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Destination_country|
+-----------------+-------------------+-----+-------------------+
|    United States|            Romania|   15|      United States|
|    United States|            Croatia|    1|      United States|
+-----------------+-------------------+-----+-------------------+
only showing top 2 rows



In [33]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "Destination_country").show(2)

+-------------------+-------------------+-----+
|Destination_country|ORIGIN_COUNTRY_NAME|count|
+-------------------+-------------------+-----+
|      United States|            Romania|   15|
|      United States|            Croatia|    1|
+-------------------+-------------------+-----+
only showing top 2 rows

